In [1]:
!pip -q install face-crop-plus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 13.2 MB/s eta 0:00:00


In [ ]:
from face_crop_plus import Cropper
from torch.cuda import is_available
from os.path import join, dirname, abspath
import cv2
import os
import shutil
import matplotlib.pyplot as plt

INPUT_DIR = "/kaggle/input/downloaded_images_2024/downloaded_images_2024"
TEMP_DIR_256_256 = "/kaggle/working/cropped_image_temp_256_256"
TEMP_DIR_NO_BLURRY = "/kaggle/working/cropped_image_temp_NO_BLURRY"
OUTPUT_DIR = "/kaggle/working/cropped_image_64_64_NO_BLURRY_Final" 

In [23]:
shutil.rmtree(OUTPUT_DIR)

In [6]:
# 第一次先進行臉的裁切照片

# Set all to False if running on CPU (unless you can wait for a bit)
TEST_QUALITY_ENHANCEMENT = False
TEST_ATTR_GROUPING = False
TEST_MASK_GROUPING = False

if __name__ == "__main__":
    # Initialize as None
    enh_threshold = None
    attr_groups = None
    mask_groups = None

    if TEST_QUALITY_ENHANCEMENT:
        enh_threshold = 0.001

    if TEST_ATTR_GROUPING:
        attr_groups = {"complete_face": [2, 3, 4, 5, 10, 11, 12, 13]}
    
    if TEST_MASK_GROUPING:
        mask_groups = {"complete_face": [2, 3, 4, 5, 10, 11, 12, 13]}

    # Initialize cropper
    cropper = Cropper(
        output_size=(256,256),
        output_format="png",
        padding="replicate",
        face_factor=0.7,
        strategy="all",
        det_threshold=0.9,
        device = "cuda:0" if is_available() else "cpu",
        enh_threshold=enh_threshold,
        attr_groups=attr_groups,
        mask_groups=mask_groups,
    )

    # Process images in the input dir and save face images to output dir
    cropper.process_dir(input_dir=INPUT_DIR, output_dir=TEMP_DIR_256_256)

Processing: 100%|██████████| 6754/6754 [1:24:06<00:00,  1.34it/s]


In [7]:
def show_image(img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()

# def is_dark(img, dark_threshold=0):
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     avg_brightness = cv2.mean(gray)[0]
#     return avg_brightness < dark_threshold

def is_blurry(img, blur_threshold=100):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    variance = cv2.Laplacian(gray, cv2.CV_64F).var()
    return variance < blur_threshold

In [17]:
def filter_and_display_images(source_folder, target_folder, brightness_threshold=100, blur_threshold=100):
    
    os.makedirs(target_folder, exist_ok=True)
    count = 0
    total_checked = 0

    image_files = [f for f in os.listdir(source_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

    for filename in image_files:
        
        image_path = os.path.join(source_folder, filename)
        img = cv2.imread(image_path)
        
        if img is None:
            print(f"⚠️ 無法讀取：{filename}")
            continue

        # dark_flag = is_dark(img, brightness_threshold)
        blur_flag = is_blurry(img, blur_threshold)

        # print("結果：")
        status = []
        # if dark_flag:
        #     status.append("太暗")
        #     print("太暗")
        if blur_flag:
            status.append("太模糊")
            # print("太模糊")
        if not status:
            status.append("好照片")
            # print("好照片")
            shutil.copy(image_path, os.path.join(target_folder, filename))
            count += 1

        total_checked += 1

    print(f"\n✅ 前 {total_checked} 張圖片處理完畢，保留了 {count} 張合格圖片於 {target_folder}")

In [18]:
filter_and_display_images(TEMP_DIR_256_256, TEMP_DIR_NO_BLURRY)


✅ 前 60618 張圖片處理完畢，保留了 33899 張合格圖片於 /kaggle/working/cropped_image_temp_NO_BLURRY


In [24]:
# 進行裁切照片

# Set all to False if running on CPU (unless you can wait for a bit)
TEST_QUALITY_ENHANCEMENT = False
TEST_ATTR_GROUPING = False
TEST_MASK_GROUPING = False

if __name__ == "__main__":
    # Initialize as None
    enh_threshold = None
    attr_groups = None
    mask_groups = None

    if TEST_QUALITY_ENHANCEMENT:
        enh_threshold = 0.001

    if TEST_ATTR_GROUPING:
        attr_groups = {"complete_face": [2, 3, 4, 5, 10, 11, 12, 13]}
    
    if TEST_MASK_GROUPING:
        mask_groups = {"complete_face": [2, 3, 4, 5, 10, 11, 12, 13]}

    # Initialize cropper
    cropper = Cropper(
        output_size=(64,64),
        output_format="png",
        padding="Constant",
        face_factor=0.7,
        strategy="all",
        det_threshold=0.95,
        device = "cuda:0" if is_available() else "cpu",
        enh_threshold=enh_threshold,
        attr_groups=attr_groups,
        mask_groups=mask_groups,
    )

    # Process images in the input dir and save face images to output dir
    cropper.process_dir(input_dir=TEMP_DIR_NO_BLURRY, output_dir=OUTPUT_DIR)

Processing: 100%|██████████| 4238/4238 [35:57<00:00,  1.96it/s]


In [ ]:
!zip -rq /kaggle/working/cropped_image_64_64_NO_BLURRY_Final.zip /kaggle/working/cropped_image_64_64_NO_BLURRY_Final